In [11]:
import torch
from torchtext import data
import pandas as pd
import numpy as np
import spacy
import sys
sys.path.append('../')
from data_util import config
if config.use_gpu:
    device = 0
else:
    device = -1

nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [15]:
# split the dataset into train and test. wikihowAll.csv is in ../data/
TEST_FRAC = 0.2
whole_dataset = pd.read_csv('../data/wikihowAll.csv')
mask = np.random.rand(len(whole_dataset)) < (1 - TEST_FRAC)
train = whole_dataset[mask]
test = whole_dataset[~mask]

In [21]:
train.to_csv('../data/train.csv', index=False)
test.to_csv('../data/test.csv', index=False)

In [12]:
def clean(text):
    text = text.replace('\n', '')
    return text.strip()

In [13]:
def tokenizer(s):
    return [w.text.lower() for w in nlp(clean(s))]

In [14]:
article_field = data.Field(sequential=True,
                           tokenize=tokenizer,
                           include_lengths=True,
                           use_vocab=True)
headline_field = data.Field(sequential=True,
                           tokenize=tokenizer,
                           include_lengths=True,
                           use_vocab=True)

In [22]:
fields = [
    ('headline', headline_field),
    ('title', None),
    ('text', article_field)
]

train_set, test_set = data.TabularDataset.splits(path='../data',
                                            format='csv',
                                            train='train.csv',
                                            validation='test.csv',
                                            fields=fields,
                                            skip_header=True)

KeyboardInterrupt: 

In [ ]:
article_field.build_vocab(train_set, test_set, max_size=config.vocab_size)
headline_field.build_vocab(train_set, test_set, max_size=config.vocab_size)

In [ ]:
train_bch, test_bch = data.BucketIterator.splits(datasets=(train_set, test_set), # specify train and validation Tabulardataset
                                            batch_sizes=(config.batch_size,config_batch_size),  # batch size of train and validation
                                            device=device, # -1 mean cpu and 0 or None mean gpu
                                            repeat=False)